# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/dennysurdiales/Desktop/python-api-challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bethel,41.3712,-73.4140,26.47,69,75,6.69,US,1686601632
1,1,Sola,-13.8833,167.5500,27.09,75,82,6.26,VU,1686601329
2,2,Ryōtsu-minato,38.0833,138.4333,20.40,86,86,0.54,JP,1686601895
3,3,La Sarre,48.8002,-79.1996,20.77,64,100,3.36,CA,1686601641
4,4,Lompoc,34.6391,-120.4579,22.05,77,100,3.09,US,1686601871


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
%%capture --no-display

# Configure the map plot


map_plot_1=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size= "Humidity",
    color="City",
    alpha=.70)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness
city_data_narrow=city_data_df[(city_data_df['Max Temp'] < 27) &
                             (city_data_df['Max Temp'] > 21) &
                             (city_data_df['Wind Speed'] < 4.5) &
                             (city_data_df['Cloudiness'] == 0)
                             ]
#drop na data
city_data_df_narrow=city_data_narrow.dropna()
# Display sample data
city_data_df_narrow

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,37,Fort McMurray,56.7268,-111.3810,21.77,40,0,3.09,CA,1686601903
148,148,Cologne,50.9333,6.9500,24.92,40,0,2.06,DE,1686601345
229,229,Revelstoke,50.9832,-118.2023,24.01,60,0,2.06,CA,1686601811
247,247,Bani Walid,31.7566,13.9942,24.47,50,0,4.32,LY,1686601954
252,252,Arraial do Cabo,-22.9661,-42.0278,22.98,94,0,3.09,BR,1686601955
325,325,Qalāt,32.1058,66.9083,24.29,14,0,2.59,AF,1686601973
385,385,Belmonte,-15.8631,-38.8828,24.20,78,0,2.44,BR,1686601612
471,471,Chibougamau,49.9168,-74.3659,25.79,34,0,3.09,CA,1686601939
492,492,Ceyhan,37.0247,35.8175,22.19,86,0,1.14,TR,1686602016
508,508,Koungou,-12.7336,45.2042,24.84,78,0,2.06,YT,1686602020


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_data_df_narrow[["City", "Country", "Lat", "Lng","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
37,Fort McMurray,CA,56.7268,-111.3810,40,
148,Cologne,DE,50.9333,6.9500,40,
229,Revelstoke,CA,50.9832,-118.2023,60,
247,Bani Walid,LY,31.7566,13.9942,50,
252,Arraial do Cabo,BR,-22.9661,-42.0278,94,
325,Qalāt,AF,32.1058,66.9083,14,
385,Belmonte,BR,-15.8631,-38.8828,78,
471,Chibougamau,CA,49.9168,-74.3659,34,
492,Ceyhan,TR,37.0247,35.8175,86,
508,Koungou,YT,-12.7336,45.2042,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:

# Set parameters to search for a hotel
radius = 10000

categories= "accommodation.hotel"
params = {
   "categories":categories,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng=hotel_df.loc[index, "Lng"]
    lat= hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Fort McMurray - nearest hotel: No hotel found
Cologne - nearest hotel: No hotel found
Revelstoke - nearest hotel: No hotel found
Bani Walid - nearest hotel: No hotel found
Arraial do Cabo - nearest hotel: No hotel found
Qalāt - nearest hotel: No hotel found
Belmonte - nearest hotel: No hotel found
Chibougamau - nearest hotel: No hotel found
Ceyhan - nearest hotel: No hotel found
Koungou - nearest hotel: No hotel found
Shahrud - nearest hotel: No hotel found
Cochrane - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
37,Fort McMurray,CA,56.7268,-111.3810,40,No hotel found
148,Cologne,DE,50.9333,6.9500,40,No hotel found
229,Revelstoke,CA,50.9832,-118.2023,60,No hotel found
247,Bani Walid,LY,31.7566,13.9942,50,No hotel found
252,Arraial do Cabo,BR,-22.9661,-42.0278,94,No hotel found
325,Qalāt,AF,32.1058,66.9083,14,No hotel found
385,Belmonte,BR,-15.8631,-38.8828,78,No hotel found
471,Chibougamau,CA,49.9168,-74.3659,34,No hotel found
492,Ceyhan,TR,37.0247,35.8175,86,No hotel found
508,Koungou,YT,-12.7336,45.2042,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
map_plot_2=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    color="City",
    scale=0.5,
    size= "Humidity",
    hover_cols = ["Hotel Name", "Country"]
    )

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)